# 문서 분류(Document Classification)

## 데이터 준비

* 문서 분류에 필요한 데이터는 `scikit-learn`이 제공하는 20개의 주제를 가지는 뉴스그룹 데이터를 사용
* 텍스트는 `CounterVectorizer`를 거쳐 DTM 행렬로 변환
* DTM 행렬은 문서에 등장하는 단어들을 빈도 수 별로 표현한 행렬


In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

news = fetch_20newsgroups()

x = news.data
y = news.target

cv = CountVectorizer()
x = cv.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(7919, 130107) (7919,) (3395, 130107) (3395,)


In [2]:
# 0번째 문서
print(x_train[0])

  (0, 56979)	2
  (0, 50527)	3
  (0, 85354)	3
  (0, 111322)	1
  (0, 37780)	1
  (0, 68532)	6
  (0, 114731)	1
  (0, 87620)	1
  (0, 95162)	1
  (0, 64095)	1
  (0, 89362)	4
  (0, 76032)	1
  (0, 28615)	1
  (0, 114579)	1
  (0, 89860)	1
  (0, 114455)	12
  (0, 115475)	3
  (0, 32311)	1
  (0, 48620)	1
  (0, 66608)	1
  (0, 51730)	1
  (0, 90252)	3
  (0, 128402)	2
  (0, 62221)	5
  (0, 35983)	1
  :	:
  (0, 107025)	1
  (0, 34507)	1
  (0, 51413)	3
  (0, 103486)	1
  (0, 61510)	1
  (0, 114262)	1
  (0, 66418)	1
  (0, 123039)	2
  (0, 41376)	1
  (0, 81163)	2
  (0, 32812)	1
  (0, 69866)	2
  (0, 116065)	1
  (0, 105971)	1
  (0, 104195)	1
  (0, 119360)	1
  (0, 101408)	1
  (0, 92246)	3
  (0, 108641)	1
  (0, 69675)	2
  (0, 52261)	3
  (0, 42780)	2
  (0, 80089)	1
  (0, 119050)	1
  (0, 85067)	1


## scikit-learn을 이용한 문서 분류

In [3]:
from sklearn.metrics import accuracy_score

### Logistic Regression

* Logistic Regression은 특성상 다중 분류에는 적합하지 않음, 이진 분류를 위한 모델

In [4]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)
pred = LR.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8606774668630339


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Naive Bayes Classification
* 베이즈 정리를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립임을 가정
* 입력 특성에 따라 3개의 분류기 존재
  * 가우시안 나이브 베이즈 분류기
  * 베르누이 나이브 베이즈 분류기
  * 다항 나이브 베이즈 분류기

#### DTM을 이용한 Naive Bayes

In [8]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(x_train, y_train)
pred = NB.predict(x_test)
acc= accuracy_score(pred, y_test)
print(acc)

0.8147275405007364


#### tf-idf를 이용한 정확도 향상

In [9]:
#Document term matrix 대신 tf-idf를 이용해 정확도 향상 가능
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.fit_transform(x_test)

NB.fit(x_train_tf, y_train)
pred = NB.predict(x_test_tf)
acc = accuracy_score(pred, y_test)
print(acc)

0.8250368188512518


### Support Vector Machine
* 회귀, 분류, 이상치 탐지 등에 사용되는 지도학습 방법
* 클래스 사이의 경계이 위치한 데이터 포인트를 Support Vector 라고 함
* 각 서포트 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지 학습
* 각 서포트 벡터 사이의 마진이 가장 큰 방향으로 학습
* 서포트 벡터까지의 거리와 서포트 벡터의 중요도를 기반으로 예측 수행

In [6]:
from sklearn import svm

SVM = svm.SVC(kernel='linear')
SVM.fit(x_train, y_train)
pred = SVM.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8238586156111929


### Decision Tree
* 분류와 회귀에 사용되는 지도학습 방법
* 데이터 특성으로부터 추론된 결정 규칙을 통해 값을 예측
* if-then-else 결정 규칙을 통해 데이터 학습
* 트리 깊이가 깊을 수록 복잡한 모델

In [10]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
pred = DT.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.6341678939617084


### XGBoost
* 트리 기반의 앙상블 기법
* 분류에 있어서 다른 알고리즘보다 좋은 예측 성능
* 기본적으로 GBM기반이지만 GBM 단점인 느린 수행 시간과 과적합 규제 부재 등의 문제를 해결

In [11]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=30,
                    learning_rate=0.05,
                    max_depth=3)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc= accuracy_score(pred, y_test)
print(acc)

0.7134020618556701


##교차 검증

* 일반 검증은 학습 데이터가 테스트 데이터로 사용되지 않음
* 교차 검증은 데이터를 n개의 집합으로 나누어 정확도를 계산해 학습 데이터로 사용된 데이터도 테스트 데이터로 사용
* 교차 검증을 사용하면 일반 검증보다 모델의 일반화가 잘 되어 있는지 평가 가능
* 앞서 구성한 나이브 베이즈 모델을 교차 검증

In [12]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(NB, x, y, cv=5)
print(scores,scores.mean())

[0.83870968 0.83826779 0.82368537 0.83031374 0.83642794] 0.833480903927519


* 교차 검증을 통해 일반 검증보다 좀 더 일반화된 모델 생성 가능
* 교차 검증은 일반 검증에 비해 n번 검증을 해 비용이 더 많이 소요

## 정밀도와 재현률 

* 정밀도(precision)는 양성 클래스(정답)으로 예측한 샘플이 양성 클래스일 확률을 의미
* 모델이 얼마나 양성 클래스를 잘 예측하는지를 나타냄
* 재현률(recall)은 양성 클래스인 샘플에서 모델이 양성 클래스로 예측한 샘플 비율을 의미하며, 모델이 얼마나 실제 상황을 재현하는지를 나타냄
* 정밀도와 재현율의 가중조화평균인 F1-score라는 지표는 정확도에 비해 더 효과적인 모델 분석 지표로 알려져 있음
* 직접 계산할 수도 있으나, scikit-learn은 이를 편리하게 계산해주는 함수를 제공

* 다중 클래스 분류 문제에서 정밀도와 재현률을 계산할 때는 클래스간의 지표를 어떻게 합칠지 지정 필요

  * None - 클래스간 지표를 합치지 말고 그대로 출력
  * micro - 정밀도와 재현률이 같음, 이로 인해 f1-score도 정밀도, 재현률과 동일
  * macro - 클래스간 지표를 단순 평균한 값
  * weighted - 클래스간 지표를 가중 평균한 값

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='micro')
recall = recall_score(pred, y_test, average='micro')
f1 = f1_score(pred, y_test, average='micro')

print(precision, recall ,f1)

0.7134020618556701 0.7134020618556701 0.7134020618556701


In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='macro')
recall = recall_score(pred, y_test, average='macro')
f1 = f1_score(pred, y_test, average='macro')

print(precision, recall ,f1)

0.7123177594016945 0.7426954181409642 0.7216491835258775


## 그리드 검색을 이용한 파라미터 최적화

* 그리드 검색을 사용하면 분류기에 사용하는 파라미터 최적화 가능
* 그리드 검색을 통해 앞서 구성한 나이브 베이즈 모델의 'alpha' 파라미터를 최적화시키는 예제

* `estimator`: 사용 모델 객체     
* `param_grid`: 사용 객체:지정 파라미터 리스트로 구성된 딕셔너리    
* `scoring`: 최적화하고자 하는 성능 지표   
* `cv`: 교차 검증 분할 개수      

In [15]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(estimator=NB, 
                  param_grid={'alpha':[0.001, 0.01, 0.1, 1.]}, 
                  scoring='accuracy', 
                  cv=10)
GS.fit(x, y)

print(GS.best_score_)
print(GS.best_params_)

0.8897820965842167
{'alpha': 0.001}
